Import package yang dibutuhkan

In [10]:
import os
import shutil
import torch
from ultralytics import YOLO
import numpy as np
import cv2
from time import time
from PIL import Image
from random import randrange

Parameter yang digunakan

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
YOLOVERSION = "yolov8s"
EPOCHS = 20
PRETRAINED=True
SIZE=640
WORKERS=2
BATCH=8
THRESHOLD=.5
# if os.path.exists('runs'):
#     shutil.rmtree('runs')  

Load model YOLOv8s

In [3]:
model = YOLO('{}.pt'.format(YOLOVERSION))
print(DEVICE)

cuda


Training model

In [4]:
model.train(data='data.yaml', epochs = EPOCHS,
                    batch=BATCH, pretrained=PRETRAINED,
                    imgsz=SIZE, workers=WORKERS)

New https://pypi.org/project/ultralytics/8.2.2 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.192  Python-3.11.9 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002130F426D90>
fitness: 0.6772895926865625
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([    0.37806,     0.80584,     0.77744,     0.76468,     0.38498,     0.84232])
names: {0: 'Goggles', 1: 'boots', 2: 'gloves', 3: 'helmet', 4: 'mask', 5: 'vest'}
plot: True
results_dict: {'metrics/precision(B)': 0.782522589406628, 'metrics/recall(B)': 0.9661334667183982, 'metrics/mAP50(B)': 0.8429065142521401, 'metrics/mAP50-95(B)': 0.6588877125126094, 'fitness': 0.6772895926865625}
save_dir: WindowsPath('runs/detect/train')
speed: {'preprocess': 0.24169840467655884, 'inference': 7.316236457811987, 'loss': 0.0013353507413034161, 'postprocess': 1.0372360931630529}

Percobaan deteksi

In [4]:
def detect(model, imgs, box_format='xyxy',th=0.5):
    total_crops = []
    total_boxes = []
    total_scores = []
    total_cls = []
    
    results = model.predict(imgs, stream=False, verbose=False)
    
    for i, image_result in enumerate(results):
        filtered_crops = []
        filtered_boxes = []
        filtered_scores = []
        filtered_cls = []
        
        img = imgs[i]
        img_w, img_h = np.array(img).shape[:2]
        
        if box_format == 'xyxy':
            image_boxes = image_result.boxes.xyxy.cpu().numpy()
        elif box_format == 'xywh':
            image_boxes = image_result.boxes.xyxy.cpu().numpy()
            image_boxes = [[round(x1), round(y1), round(np.abs(x1-x2)), round(np.abs(y1-y2))] for x1,y1,x2,y2 in image_boxes]
        elif box_format == 'xyxyn':
            image_boxes = image_result.boxes.xyxyn.cpu().numpy()
        elif box_format == 'xywhn':
            image_boxes = image_result.boxes.xyxyn.cpu().numpy()
            image_boxes = [[float(x1), float(y1), float(np.abs(x1-x2)), float(np.abs(y1-y2))] for x1,y1,x2,y2 in image_boxes]
              
        image_scores = image_result.boxes.conf.cpu().numpy()
        image_cls = image_result.boxes.cls.cpu().numpy()
        
        for j in range(len(image_boxes)):
            (x1, y1, x2, y2), score, c = image_boxes[j], image_scores[j], image_cls[j]
            if score >= th:
                filtered_scores.append(score)
                filtered_cls.append(c)
                if box_format == 'xyxy':
                    filtered_crops.append(img.crop([x1,y1,x2,y2]))
                    filtered_boxes.append([int(x1),int(y1),int(x2),int(y2)])
                elif box_format == 'xyxyn':
                    filtered_crops.append(img.crop([x1*img_w, y1*img_h, x2*img_w, y2*img_h]))
                    filtered_boxes.append([float(x1),float(y1),float(x2),float(y2)])
                elif box_format == 'xywh':
                    filtered_crops.append(img.crop([x1,y1,x1+x2,y1+y2]))
                    filtered_boxes.append([int(x1),int(y1),int(x2),int(y2)])
                elif box_format == 'xywhn':
                    filtered_crops.append(img.crop([x1*img_w,y1*img_h,(x1+x2)*img_w,(y1+y2)*img_h]))
                    filtered_boxes.append([float(x1),float(y1),float(x2),float(y2)])
                    
        total_crops.append(filtered_crops)        
        total_boxes.append(filtered_boxes)
        total_scores.append(filtered_scores)
        total_cls.append(filtered_cls)
                

    return total_crops, total_boxes, total_scores, total_cls

In [19]:
model = YOLO('./runs/detect/train/weights/best.pt')
labels = ['Goggles', 'boots', 'gloves', 'helmet', 'mask', 'vest']

In [28]:
image = cv2.imread("image_211_jpg.rf.d3903a253abcc558769bf013dd40d51c.jpg")

crops, boxes, scores, cls = detect(model, [Image.fromarray(image)], box_format='xywh',th=THRESHOLD)

for (left, top, width, height), score, label in zip(boxes[0], scores[0], cls[0]):
    color = (randrange(256), randrange(256), randrange(256))
    cv2.rectangle(image, (int(left), int(top)), (int(left+width), int(top+height)+5), color, 2)
    cv2.putText(image, labels[int(label)], (int(left), int(top)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

cv2.imshow("Deteksi Gambar", image)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [27]:
image = cv2.imread("Video4_280_jpg.rf.78840e232ee262d150d0965508765ebc.jpg")

crops, boxes, scores, cls = detect(model, [Image.fromarray(image)], box_format='xywh',th=THRESHOLD)

for (left, top, width, height), score, label in zip(boxes[0], scores[0], cls[0]):
    color = (randrange(256), randrange(256), randrange(256))
    cv2.rectangle(image, (int(left), int(top)), (int(left+width), int(top+height)+5), color, 2)
    cv2.putText(image, labels[int(label)], (int(left), int(top)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

cv2.imshow("Deteksi Gambar", image)

cv2.waitKey(0)
    
cv2.destroyAllWindows()

In [31]:
cap = cv2.VideoCapture("Video-Contoh-Trim.mp4")
while True:
    ret, image = cap.read()
    crops, boxes, scores, cls = detect(model, [Image.fromarray(image)], box_format='xywh',th=0.2)

    for (left, top, width, height), score, label in zip(boxes[0], scores[0], cls[0]):
        color = (randrange(256), randrange(256), randrange(256))
        cv2.rectangle(image, (int(left), int(top)), (int(left+width), int(top+height)+5), color, 2)
        cv2.putText(image, labels[int(label)], (int(left), int(top)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    image = cv2.resize(image, (1280, 720))
    cv2.imshow("Deteksi Gambar", image)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

Apabila dilihat dari video, tidak terdapat hasil dari model. Hal ini kemungkinan karena model tidak cukup bagus untuk melakukan deteksi pada jarak yang jauh. Solusi yang bisa dilakukan yaitu menambahkan dataset untuk kondisi jarak jauh seperti video, dan juga bisa dengan menggunakan model yang lebih besar seperti YOLOv8l ataupun YOLOv8x